# PDE vs MC vs BSM – Vanilla European Option Pricing

This notebook benchmarks European call option pricing using three numerical and analytical methods:

- ✅ **Black-Scholes Closed Form** (Analytical)
- ✅ **Monte Carlo Simulation** (Euler-Maruyama)
- ✅ **Crank-Nicolson Finite Difference** (PDE)

We use identical inputs and compare accuracy and stability of each technique.

In [1]:
S0 = 100        # Spot price
K = 100         # Strike price
r = 0.05        # Risk-free rate
sigma = 0.2     # Volatility
T = 1.0         # Time to maturity

option_type = "call"

In [2]:
from scipy.stats import norm
import numpy as np

def bsm_price(S, K, r, sigma, T, option_type="call"):
    if T == 0:
        return max(0, (S - K) if option_type == "call" else (K - S))
    d1 = (np.log(S/K) + (r + 0.5*sigma**2)*T) / (sigma*np.sqrt(T))
    d2 = d1 - sigma*np.sqrt(T)
    if option_type == "call":
        return S * norm.cdf(d1) - K * np.exp(-r*T) * norm.cdf(d2)
    else:
        return K * np.exp(-r*T) * norm.cdf(-d2) - S * norm.cdf(-d1)

bsm = bsm_price(S0, K, r, sigma, T, option_type)
print(f"Black-Scholes Price: {bsm:.4f}")

Black-Scholes Price: 10.4506


In [3]:
def mc_price_european(S0, K, r, sigma, T, M=100_000, N=100, option_type="call", seed=42):
    np.random.seed(seed)
    dt = T / N
    S = np.zeros((M, N + 1))
    S[:, 0] = S0
    for t in range(1, N + 1):
        Z = np.random.normal(size=M)
        S[:, t] = S[:, t-1] * np.exp((r - 0.5*sigma**2)*dt + sigma*np.sqrt(dt)*Z)

    ST = S[:, -1]
    payoff = np.maximum(ST - K, 0) if option_type == "call" else np.maximum(K - ST, 0)
    price = np.exp(-r * T) * np.mean(payoff)
    return price

mc = mc_price_european(S0, K, r, sigma, T)
print(f"Monte Carlo Price: {mc:.4f}")

Monte Carlo Price: 10.4231


In [4]:
def crank_nicolson(S0, K, r, sigma, T, Smax=200, M=100, N=100, option_type="call"):
    dt = T / N
    dS = Smax / M
    S = np.linspace(0, Smax, M+1)
    V = np.maximum(S - K, 0) if option_type == "call" else np.maximum(K - S, 0)

    alpha = 0.25 * dt * (sigma**2 * (np.arange(M+1)**2) - r * np.arange(M+1))
    beta  = -dt * 0.5 * (sigma**2 * (np.arange(M+1)**2) + r)
    gamma = 0.25 * dt * (sigma**2 * (np.arange(M+1)**2) + r * np.arange(M+1))

    A = np.zeros((M-1, M-1))
    B = np.zeros((M-1, M-1))
    for i in range(1, M):
        if i > 1:
            A[i-1, i-2] = -alpha[i]
            B[i-1, i-2] =  alpha[i]
        A[i-1, i-1] = 1 - beta[i]
        B[i-1, i-1] = 1 + beta[i]
        if i < M-1:
            A[i-1, i] = -gamma[i]
            B[i-1, i] =  gamma[i]

    from scipy.linalg import solve
    for j in range(N):
        V[1:M] = solve(A, B @ V[1:M])

    return np.interp(S0, S, V)

pde = crank_nicolson(S0, K, r, sigma, T)
print(f"PDE Crank-Nicolson Price: {pde:.4f}")

PDE Crank-Nicolson Price: 10.3537


### Pricing Comparison Summary

| Method            | Price     |
|-------------------|-----------|
| Black-Scholes     |       |
| Monte Carlo       | XYZ       |
| Crank-Nicolson PDE| XYZ       |     

This confirms the **numerical stability and accuracy** of the PDE and MC methods when benchmarked against analytical BSM pricing.

Discrepancies can be attributed to:
- Time/space grid size (PDE)
- Number of paths (MC)